In [66]:
import pandas as pd
import numpy as np
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import  f1_score
import random
np.set_printoptions(threshold=np.inf)

In [67]:
cols=feature=[#'IP_id',
#'ICMP_chksum',
#'ICMP_id',
#'TCP_seq',
#'TCP_ack',
 #'TCP_chksum',
# 'UDP_chksum',
# 'DNS_id',
# 'BOOTP_xid',
'sport_class',
'dport_class',
#'sport23',
#'dport23',
#'sport_bare',
# 'dport_bare',
# 'TCP_sport',
# 'TCP_dport',
# 'UDP_sport',
# 'UDP_dport',
 'pck_size',
 'Ether_type',
 'LLC_ssap',
 'LLC_ctrl',
 'EAPOL_version',
 'EAPOL_type',
 'EAPOL_len',
 'IP_version',
 'IP_ihl',
 'IP_tos',
 'IP_len',
 'IP_flags',
 'IP_DF',
 'IP_ttl',
 'IP_proto',
 'IP_options',
 'ICMP_type',
 'ICMP_code',
 'ICMP_seq',
 'TCP_dataofs',
 'TCP_flags',
 'TCP_FIN',
 'TCP_SYN',
 'TCP_RST',
 'TCP_PSH',
 'TCP_ACK',
 'TCP_window',
 'TCP_options',
 'UDP_len',
 'DHCP_options',
 'BOOTP_op',
 'BOOTP_htype',
 'BOOTP_hlen',
 'BOOTP_secs',
 'BOOTP_flags',
 'BOOTP_sname',
 'BOOTP_file',
 'BOOTP_options',
 'DNS_qr',
 'DNS_aa',
 'DNS_rd',
 'DNS_ra',
 'DNS_rcode',
 'DNS_qdcount',
 'DNS_ancount',
 'DNS_nscount',
 'DNS_arcount',
 'payload_bytes',    
 'entropy',
 'Protocol',
#"MAC",
 'Label']

In [68]:
print("INIZIA GA")

test='/path/to/Aalto_test_IoTDevID.csv'
train='/path/to/Aalto_train_IoTDevID.csv'

# TRAIN
df = pd.read_csv(train,usecols=cols)#,keep_default_na=False)#, na_values='')#,header=None )
df.replace(np.nan, 0.0, inplace=True)
X_train =df[df.columns[0:-1]]
df[df.columns[-1]] = df[df.columns[-1]].astype('category')
y_train=df[df.columns[-1]].cat.codes  


# TEST
df = pd.read_csv(test,usecols=cols)#,na_values="NaN")#,header=None )
df.replace(np.nan, 0.0, inplace=True)
X_test =df[df.columns[0:-1]]
df[df.columns[-1]] = df[df.columns[-1]].astype('category')
y_test=df[df.columns[-1]].cat.codes #associa un numero per ogni categoria (label) trovata prima

logmodel = DecisionTreeClassifier()

INIZIA GA


In [69]:
#defining various steps required for the genetic algorithm
# GA adapted from https://datascienceplus.com/genetic-algorithm-in-machine-learning-using-python/

#Inizializzo la popolazione in modo random
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=bool)
        chromosome[:int(0.3*n_feat)]=False
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population

In [70]:
#funzione che calcolerà il valore di fitness di ogni individuo
def fitness_score(population):
    scores = []
    i=0
    for chromosome in population: #pop insieme di array con true/false // cromo= singolo array con true e false
        #X_train.iloc[:,chromosome] -> prende i valori del dataset solo delle colonne con True nel cromo
        #y_train -> array 1D con la label dei record del dataset
        logmodel.fit(X_train.iloc[:,chromosome],y_train)
        predictions = logmodel.predict(X_test.iloc[:,chromosome])
        scores.append(f1_score(y_test,predictions,average= "macro"))
        i+=1
    scores, population = np.array(scores), np.array(population)
    inds = np.argsort(scores)   #ordina elem in base al valore di scores
    return list(scores[inds][::-1]), list(population[inds,:][::-1])

In [71]:
def roulette_selection(population_after_fit, scores, size):
    #NORMALIZZO I DATI
    score_sum = sum(scores)
    selection_probabilities = [score/score_sum for score in scores]
    i = size-1#len(selection_probabilities)-1
    cumulative_probabilities=[0.0]*size

    while i>-1:
        cumulative_probabilities[i] = sum(selection_probabilities[i:])
        i-=1

    rnd = random.random()

    for j, c_prob in enumerate(cumulative_probabilities):
        if rnd<=c_prob:
            if population_after_fit[j] is not None:
                return population_after_fit[j]

In [72]:
def single_point_crossover(parent1, parent2, pop_after_fit):
    parent1_len = len(parent1)
    parent2_len = len(parent2)
    max_len = max(parent1_len, parent2_len)
    parent1 = list(parent1) + [0] * (max_len - parent1_len)
    parent2 = list(parent2) + [0] * (max_len - parent2_len)
    crossover_point = random.randint(1, max_len - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    pop_after_fit.append(child1)
    pop_after_fit.append(child2)
    return pop_after_fit

In [73]:
#aggiunge elem stocastici per non rimanere incastrati in una zona viene modificato un cromosoma
def mutation(pop_after_cross,mutation_rate):
    population_nextgen = []
    for i in range(0,len(pop_after_cross)):
        chromosome = pop_after_cross[i]
        for j in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[j]= not chromosome[j]
        population_nextgen.append(chromosome)
    #print(population_nextgen)
    return population_nextgen

In [74]:
def generations_dd(size,n_feat,mutation_rate,n_gen,X_train,
                X_test, y_train, y_test):

    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    num_iter = 0
    for i in range(n_gen):
        second=time.time()
        scores, pop_after_fit = fitness_score(population_nextgen)
        tempo=time.time()-second
        parent1 = roulette_selection(pop_after_fit, scores, size)
        parent2 = roulette_selection(pop_after_fit, scores, size)
        pop_after_cross = single_point_crossover(parent1, parent2, pop_after_fit)
        population_nextgen = mutation(pop_after_cross,mutation_rate)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
        print ('%-30s %-30s %-30s %-30s ' % (num_iter, np.mean(scores),np.std(scores),tempo))
        num_iter += 1
    return best_chromo,best_score

In [76]:
print ('%-30s %-30s %-30s %-30s ' % ("NUM. GEN", "MEAN","STD","TIME"))
chromo,score=generations_dd(size=50,n_feat=52,mutation_rate=0.005,
                         n_gen=20,X_train=X_train,X_test=X_test,y_train=y_train,y_test=y_test)

NUM. GEN                       MEAN                           STD                            TIME                           
0                              0.721380828045207              0.06800858460979657            6.879462718963623              
1                              0.7232375523343493             0.06748309466403427            6.988962888717651              
2                              0.7221961715090396             0.07393046849313617            7.2590172290802                
3                              0.7238693664310707             0.07314194496689713            7.488349199295044              
4                              0.722970188769351              0.07432154920744291            7.751039505004883              
5                              0.7251748991443502             0.07355248244122811            8.242187023162842              
6                              0.7272293443824807             0.07322618645031834            8.326710939407349              


In [77]:
score[0]

0.7886340696473111

In [78]:
chromo[0]

array([ True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True, False, False,  True, False,  True,  True,
        True,  True, False, False,  True, False, False,  True,  True,
        True,  True, False,  True,  True, False, False, False,  True,
        True, False,  True,  True,  True,  True,  True])